# 1. Importing Libraries

In [1]:
import cv2 #opencv
import numpy as np
import os   #helps with path
from matplotlib import pyplot as plt #to use plt.imshow()
import time              #to measure time between frames 
import mediapipe as mp   


# 2 . drawingutil and hands module

In [2]:
mpDraw=mp.solutions.drawing_utils
mpHands=mp.solutions.hands

# 3. Function to detect points on hands and then drawing on the hands

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB because by default opencv use bgr but we need rgb for mediapipe to process image
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_landmarks(image, results):
    if results.multi_hand_landmarks:
        for num, handsLms in enumerate(results.multi_hand_landmarks):
            mpDraw.draw_landmarks(image,handsLms, mpHands.HAND_CONNECTIONS,
                             mpDraw.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mpDraw.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2) )

# 4. function to get handedness ie left or right and to extract point to numpy array

In [5]:
def get_label(index,results):
    label = None
    for idx, classification in enumerate(results.multi_handedness):
        if classification.classification[0].index == index:
            label = classification.classification[0].label
#         print(index,idx,label)
    if label:
        return label
    else:
        if index == 1:
            return get_label(0,results)
        elif index == 0:
            return get_label(1,results)
        else:
            return label
def extract_keypoints(results):
    hands = [np.zeros(21*3),np.zeros(21*3)]
    if results.multi_hand_landmarks:
        for num, handsLms in enumerate(results.multi_hand_landmarks):
            label = get_label(num,results)
            if label == 'Right':
                hands[0] = np.array([[res.x, res.y, res.z] for res in handsLms.landmark]).flatten()
            if label == 'Left':
                hands[1] = np.array([[res.x, res.y, res.z] for res in handsLms.landmark]).flatten()
            
    return np.concatenate(hands)

# 5. Number of videos of dataset and videolength

In [6]:
# Thirty videos worth of data
no_videos = 400

# Videos are going to be 15 frames in length
video_length = 15

# 6. defining path of dataset and also path where we will store the processed data ie folder Edata in current directory

dataset link <a href="https://drive.google.com/drive/folders/1RZaXXy3pr7YLSv1jKFXDgmNC2nzkgV1P?usp=sharing">DATASET<a>

In [8]:
cwd = os.getcwd()
print(cwd)

C:\Users\91963\Desktop\HandSignRecognition\DEMO\newmodel\AJmodel


In [9]:
datapath = os.path.join('../../KgData/asl4g/train/')
os.listdir(datapath)

['A',
 'B',
 'C',
 'D',
 'del',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'nothing',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'space',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z']

In [7]:
#path for numpy array of data
DATA_PATH = os.path.join('AtoZdata')

In [11]:
os.listdir(cwd)

['.ipynb_checkpoints',
 '26.04.2022_17.30.00_REC.mp4',
 '26.04.2022_17.54.20_Latest.mp4',
 'ATOZ',
 'ATOZ.zip',
 'AtoZdata',
 'AtoZdataDistance',
 'board1.jpg',
 'HandGestureRecognitionNeuralNetworkA-J-v1-lstm.ipynb',
 'HandGestureRecognitionNeuralNetworkA-J-v1.ipynb',
 'HandGestureRecognitionNeuralNetworkA-J-v2-distance-included.ipynb',
 'HandGestureRecognitionNeuralNetworkAllDataButSingleFrame.ipynb',
 'LatestWorking.zip',
 'model26April.h5',
 'model27AprilDist.h5',
 'model27Aprillstm.h5',
 'model27Aprillstm2.h5',
 'model31march.h5',
 'New folder',
 'runner.ipynb',
 'trainLogsdNeuralNetworkATOZ26April',
 'trainLogsdNeuralNetworkATOZ27AprilDist',
 'trainLogsdNeuralNetworkATOZ27AprilLstm',
 'trainLogsdNeuralNetworkATOZ27AprilLstm2']

# 7. defining the categories of signs 

In [8]:
categories = np.array(['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z','del','space','nothing'])

# 8. making directory to store the processed data in EData folder

In [85]:
for category in categories: 
    for i in range(no_videos):
        try: 
            os.makedirs(os.path.join(DATA_PATH,category, str(i)))
        except:
            pass

In [86]:
# os.walk('Edata')

# 9. checking data is properly fetching  and also the camera feed

In [87]:
for category in categories:
    path = os.path.join(datapath,category)
    for imgpath in os.listdir(path):
        img_array = cv2.imread(os.path.join(path,imgpath))
        img_array = cv2.resize(img_array,(480,640))
        img_array2 = cv2.flip(img_array,1)
        cv2.imshow("frame",img_array)
        if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
cv2.destroyAllWindows()

In [193]:
cap = cv2.VideoCapture(0) #creating video capture object
## Set mediapipe model

while cap.isOpened():
    #reading feed current frame
    ret,frame = cap.read() 
    flipframe = cv2.flip(frame,1)
    cv2.imshow("Open Cv1",flipframe) 
#     cv2.imshow('o2',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'): #wait for 1 milli second to check if q is pressed on keyboard
        break 
cap.release() #release our webcame
cv2.destroyAllWindows()

# 10. Extracting keypoints using extract_keypoint funtion and for every frame   #  for every category we are creating 400 videos and every video contains 15 frames.

In [88]:
with mpHands.Hands() as model:
    for category in categories:
        path = os.path.join(datapath,category)
        adl = os.listdir(path)
        cdl = 0
        for i in range(no_videos):
            b = False
            for j in range(video_length):
                image = cv2.imread(os.path.join(path,adl[cdl]))
#                 image = cv2.flip(image,1)
                image = cv2.resize(image,(640,480))
                image, results = mediapipe_detection(image, model)
                draw_landmarks(image, results)
#                 print(results.multi_handedness)
                cdl += 1
                cv2.putText(image,'f no {} Vno {} c {}'.format(j,i,category), (10,20), 
                                   cv2.FONT_HERSHEY_SIMPLEX,0.5, (150, 0, 255), 1, cv2.LINE_AA)
                cv2.imshow("frame",image)
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, category, str(i), str(j))
                np.save(npy_path, keypoints)
                
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    b = True
                    break
            if b:
                break


    cv2.destroyAllWindows()

# 11. converting categories using one hot encoding and splitting data into train and test 

In [9]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
label_map = {label:num for num, label in enumerate(categories)}

In [10]:
label_map

{'A': 0,
 'B': 1,
 'C': 2,
 'D': 3,
 'E': 4,
 'F': 5,
 'G': 6,
 'H': 7,
 'I': 8,
 'J': 9,
 'K': 10,
 'L': 11,
 'M': 12,
 'N': 13,
 'O': 14,
 'P': 15,
 'Q': 16,
 'R': 17,
 'S': 18,
 'T': 19,
 'U': 20,
 'V': 21,
 'W': 22,
 'X': 23,
 'Y': 24,
 'Z': 25,
 'del': 26,
 'space': 27,
 'nothing': 28}

In [11]:
videos, labels = [], []
for category in categories:
    for video in range(no_videos):
        window = []
        for frame_num in range(video_length):
            res = np.load(os.path.join(DATA_PATH, category, str(video), "{}.npy".format(frame_num)))
            window.append(res)
        videos.append(window)
        labels.append(label_map[category])

In [18]:
X = np.array(videos)

In [19]:
X.shape

(11600, 15, 126)

In [20]:
y = to_categorical(labels).astype(int) #one hot encoding to convert categorial variable

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [22]:
y_test.shape

(2320, 29)

In [23]:
X_train.shape

(9280, 15, 126)

In [105]:
# nsamples, nx, ny = X_train.shape
# X_train = X_train.reshape((nsamples,nx*ny))



In [110]:
# nsamples, nx, ny = X_test.shape
# X_test = X_test.reshape((nsamples,nx*ny))

# Random Forest Model

In [22]:
from sklearn.ensemble import RandomForestClassifier


In [112]:
classifier_rf = RandomForestClassifier(random_state=42, n_jobs=-1, max_depth=5,
                                       n_estimators=1000, oob_score=True)

In [113]:
classifier_rf.fit(X_train, y_train)

RandomForestClassifier(max_depth=5, n_estimators=1000, n_jobs=-1,
                       oob_score=True, random_state=42)

In [114]:
X_train.shape

(9280, 1890)

In [115]:
classifier_rf.oob_score_

0.9660077288941732

In [116]:
classifier_rf.score(X_test,y_test)

0.014655172413793103

In [117]:
rf = RandomForestClassifier(random_state=42, n_jobs=-1)

In [ ]:
params = {
    'max_depth': [2,3,5,10,20,30,40,50,60],
    'min_samples_leaf': [5,10,20,50,100,200,],
    'n_estimators': [10,25,30,50,100,200]
}

# 12. Building model and training

In [26]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,Flatten
from tensorflow.keras.callbacks import TensorBoard

In [27]:
import tensorflow as tf
ACCURACY_THRESHOLD = 0.95
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        # print(logs.get('acc'))
        # print(logs.get('categorical_accuracy'))
        if(logs.get('categorical_accuracy') > ACCURACY_THRESHOLD):
            # print(logs.get('acc'))
            # print(logs.get('categorical_accuracy'))
            print("\nReached %2.2f%% accuracy, so stopping training!!" %(ACCURACY_THRESHOLD*100))
            self.model.stop_training = True

# Instantiate a callback object
callbacks = myCallback()

In [28]:
log_dir = os.path.join('trainLogsdNeuralNetworkATOZ26Aprilimproved')
tb_callback = TensorBoard(log_dir=log_dir)

In [29]:
mycallbacks = [callbacks,tb_callback]

In [30]:
model = Sequential()
model.add(Dense(1024,activation='relu',input_shape=(15,126)))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Flatten())
model.add(Dense(29, activation='softmax'))
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [59]:
 model.fit(X_train, y_train, epochs=500, callbacks=mycallbacks)

Epoch 1/500
290/290 [==============================] - 5s 16ms/step - loss: 2.1595 - categorical_accuracy: 0.3152
Epoch 2/500
290/290 [==============================] - 4s 15ms/step - loss: 1.0434 - categorical_accuracy: 0.6571
Epoch 3/500
290/290 [==============================] - 4s 15ms/step - loss: 0.8437 - categorical_accuracy: 0.7238
Epoch 4/500
290/290 [==============================] - 4s 15ms/step - loss: 0.7162 - categorical_accuracy: 0.7678
Epoch 5/500
290/290 [==============================] - 4s 14ms/step - loss: 0.7075 - categorical_accuracy: 0.7748
Epoch 6/500
290/290 [==============================] - 4s 13ms/step - loss: 0.6423 - categorical_accuracy: 0.8012
Epoch 7/500
290/290 [==============================] - 4s 13ms/step - loss: 0.6145 - categorical_accuracy: 0.8115
Epoch 8/500
290/290 [==============================] - 3s 12ms/step - loss: 0.5755 - categorical_accuracy: 0.8211
Epoch 9/500
290/290 [==============================] - 3s 12ms/step - loss: 0.5806 - cat

290/290 [==============================] - 4s 12ms/step - loss: 0.2854 - categorical_accuracy: 0.9308
Epoch 70/500
290/290 [==============================] - 4s 12ms/step - loss: 0.2997 - categorical_accuracy: 0.9287
Epoch 71/500
290/290 [==============================] - 4s 12ms/step - loss: 0.3072 - categorical_accuracy: 0.9239 0s - loss: 0.3127 - categorical_accu
Epoch 72/500
290/290 [==============================] - 4s 12ms/step - loss: 0.2838 - categorical_accuracy: 0.9331
Epoch 73/500
290/290 [==============================] - 4s 12ms/step - loss: 0.2662 - categorical_accuracy: 0.9358
Epoch 74/500
290/290 [==============================] - 4s 12ms/step - loss: 0.3144 - categorical_accuracy: 0.9213
Epoch 75/500
290/290 [==============================] - 4s 12ms/step - loss: 0.3198 - categorical_accuracy: 0.9249
Epoch 76/500
290/290 [==============================] - 4s 12ms/step - loss: 0.2629 - categorical_accuracy: 0.9376
Epoch 77/500
290/290 [==============================] - 

290/290 [==============================] - 4s 14ms/step - loss: 0.2857 - categorical_accuracy: 0.9346
Epoch 140/500
290/290 [==============================] - 4s 12ms/step - loss: 0.2450 - categorical_accuracy: 0.9421
Epoch 141/500
290/290 [==============================] - 4s 13ms/step - loss: 0.2403 - categorical_accuracy: 0.9427
Epoch 142/500
290/290 [==============================] - 4s 13ms/step - loss: 0.3492 - categorical_accuracy: 0.9216
Epoch 143/500
290/290 [==============================] - 4s 14ms/step - loss: 0.2409 - categorical_accuracy: 0.9423
Epoch 144/500
290/290 [==============================] - 4s 13ms/step - loss: 0.2288 - categorical_accuracy: 0.9460
Epoch 145/500
290/290 [==============================] - 4s 13ms/step - loss: 0.2268 - categorical_accuracy: 0.9463
Epoch 146/500
290/290 [==============================] - 4s 13ms/step - loss: 0.2264 - categorical_accuracy: 0.9463
Epoch 147/500
290/290 [==============================] - 4s 13ms/step - loss: 0.2263 -

KeyboardInterrupt: 

In [31]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 15, 1024)          130048    
_________________________________________________________________
dense_1 (Dense)              (None, 15, 512)           524800    
_________________________________________________________________
dense_2 (Dense)              (None, 15, 256)           131328    
_________________________________________________________________
dense_3 (Dense)              (None, 15, 128)           32896     
_________________________________________________________________
dense_4 (Dense)              (None, 15, 64)            8256      
_________________________________________________________________
flatten (Flatten)            (None, 960)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 29)                2

# 13. Make prediction 

In [39]:
res = model.predict(X_test)

In [40]:
categories[np.argmax(res[2])]

'space'

In [41]:
categories[np.argmax(y_test[2])]

'space'

# 14. Save Model

In [64]:
# model.save('model27improved2.h5')

# Load Model

In [48]:
model.load_weights('model27improved2.h5')

# 15. Evaluation using Confusion Matrix and Accuracy


In [49]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [50]:
yhat = model.predict(X_test)

In [51]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [52]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[2226,    0],
        [   8,   86]],

       [[2243,    0],
        [   1,   76]],

       [[2240,    2],
        [   5,   73]],

       [[2226,    0],
        [   1,   93]],

       [[2261,    1],
        [   3,   55]],

       [[2235,    0],
        [   0,   85]],

       [[2260,    0],
        [   0,   60]],

       [[2240,    2],
        [   1,   77]],

       [[2242,    4],
        [   8,   66]],

       [[2232,    1],
        [   9,   78]],

       [[2244,    4],
        [   1,   71]],

       [[2233,    0],
        [   0,   87]],

       [[2225,    9],
        [  11,   75]],

       [[2241,    4],
        [  15,   60]],

       [[2227,    0],
        [   9,   84]],

       [[2242,    0],
        [  13,   65]],

       [[2249,    0],
        [   4,   67]],

       [[2225,    2],
        [  18,   75]],

       [[2242,    5],
        [   6,   67]],

       [[2238,    0],
        [   1,   81]],

       [[2231,   12],
        [  11,   66]],

       [[2230,    4],
        [   

In [53]:
accuracy_score(ytrue, yhat)

0.9245689655172413

# 16. Testing in real Time

In [54]:
colors = [(245,117,16), (117,245,16), (16,117,245),(166,107,245),(106,167,245),(106,117,205),(161,107,25),(130,187,245),(106,117,205),(106,100,245),(196,107,245), (117,245,16), (16,117,245),(245,117,16), (117,245,16), (16,117,245),(166,107,245),(106,167,245),(106,117,205),(161,107,25),(130,187,245),(106,117,205),(106,100,245),(196,107,245), (117,245,16), (16,117,245),(245,117,16), (117,245,16), (16,117,245)]
# def prob_viz(res, actions, input_frame, colors):
#     output_frame = input_frame.copy()
#     for num, prob in enumerate(res):
#         if prob > 90:
#             cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
#             cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2, cv2.LINE_AA)
#     return output_frame

In [170]:
def prob_viz(res, actions,input_frame):
    output_frame = input_frame.copy()
    if res[np.argmax(res)] > 0.9:
        cv2.putText(output_frame,actions[np.argmax(res)], (200,300), cv2.FONT_HERSHEY_SIMPLEX,4, (0, 0, 255), 2, cv2.LINE_AA)
    return output_frame

In [171]:
back = cv2.imread('board1.jpg')
back = cv2.resize(back,(600,600))
# cv2.imshow('back',back)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [172]:
categoriesLatest = np.array(['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z','d',' ','n'])

In [173]:
# # 1. New detection variables mode1
# import time
# sequence = []
# sentence = []
# threshold = 0.9

# cap = cv2.VideoCapture(0)
# started = False
# # Set mediapipe model 
# with mpHands.Hands() as hand:
#     i = 20
#     j = 26
#     back2 = cv2.imread('board1.jpg')
#     back2 = cv2.resize(back,(600,600))
#     while cap.isOpened():
#         current_time = time.time()
#         board = back
#         # Read feed
#         ret, frame = cap.read()
#         # Make detections
#         image, results = mediapipe_detection(frame,hand)

        
#         # Draw landmarks
#         draw_landmarks(image, results)
#         # 2. Prediction logic
#         keypoints = extract_keypoints(results)

#         sequence.append(keypoints)
# #         print(sequence)
#         sequence = sequence[-15:]
# #         print(sequence)
        
#         if len(sequence) == 15:
#             res = model.predict(np.expand_dims(sequence, axis=0))[0]
#         #3. Viz logic
#             if res[np.argmax(res)] > threshold: 
#                 casl = categoriesLatest[np.argmax(res)]
#                 if len(sentence) <= 0 or ( casl != 'n' and casl != sentence[-1]):
#                     cv2.putText(back2,casl, (i,j), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
#                     if i <= 20 and  started:
#                         j = j + 26
#                     i = (i + 20)%500
#                     if i == 0:
#                         i = 20
#                     started = True
#                 if len(sentence) > 0: 
#                     if categories[np.argmax(res)] != sentence[-1]:
#                         sentence.append(categoriesLatest[np.argmax(res)])
#                 else:
#                     sentence.append(categoriesLatest[np.argmax(res)]) 

#             if len(sentence) > 10: 
#                 sentence = sentence[-20:]

#             # Viz probabilities
#             board = prob_viz(res, categories,board)
            
        
#         # Show to screen
#         cv2.imshow('OpenCV Feed', image)
#         cv2.imshow('viz',board)
#         cv2.imshow('Text Converter',back2)

#         # Break gracefully
#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break

#     cap.release()
#     cv2.destroyAllWindows()

In [65]:
# 1. New detection variables mode 2
import time
sequence = []
sentence = []
threshold = 0.9
time_frame = []
cap = cv2.VideoCapture(0)
started = False
# Set mediapipe model 
with mpHands.Hands() as hand:
    i = 20
    j = 26
    back2 = cv2.imread('board1.jpg')
    back2 = cv2.resize(back,(600,600))
    while cap.isOpened():
        
        board = back
        # Read feed
        ret, frame = cap.read()
        # Make detections
        image, results = mediapipe_detection(frame,hand)

        
        # Draw landmarks
        draw_landmarks(image, results)
        # 2. Prediction logic
        keypoints = extract_keypoints(results)

        sequence.append(keypoints)
#         print(sequence)
        sequence = sequence[-15:]
#         print(sequence)
        
        if len(sequence) == 15:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                casl = categoriesLatest[np.argmax(res)]
                if casl != 'n' and (cv2.waitKey(1) & 0xFF == ord('v')) :
                    cv2.putText(back2,casl, (i,j), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                    if i <= 20 and  started:
                        j = j + 26
                    i = (i + 20)%500
                    if i == 0:
                        i = 20
                    started = True

            # Viz probabilities
            board = prob_viz(res, categories,board)
        # Show to screen
        cv2.imshow('OpenCV Feed', image)
        cv2.imshow('viz',board)
        cv2.imshow('Text Converter',back2)

        # Break gracefully
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [1]:
# # 1. New detection variables

# sequence = []
# sentence = []
# threshold = 0.90

# cap = cv2.VideoCapture(0)
# started = False
# # Set mediapipe model
# pred_count = {'count':0,'symbol':'n'}
# prev_symbol = 'n'
# with mpHands.Hands() as hand:
#     i = 20
#     j = 26
#     back2 = cv2.imread('board1.jpg')
#     back2 = cv2.resize(back,(600,600))
#     while cap.isOpened():
        
#         board = back
#         # Read feed
#         ret, frame = cap.read()
#         # Make detections
#         image, results = mediapipe_detection(frame,hand)

        
#         # Draw landmarks
#         draw_landmarks(image, results)
#         # 2. Prediction logic
#         keypoints = extract_keypoints(results)

#         sequence.append(keypoints)
# #         print(sequence)
#         sequence = sequence[-15:]
# #         print(sequence)
        
#         if len(sequence) == 15:
#             res = model.predict(np.expand_dims(sequence, axis=0))[0]
#         #3. Viz logic
#             if res[np.argmax(res)] > threshold: 
#                 casl = categoriesLatest[np.argmax(res)]
#                 if casl == pred_count['symbol']:
#                     pred_count['count'] += 1
#                 else:
#                     pred_count['symbol'] = casl
#                     pred_count['count'] = 0
#                 if ( casl != 'n'  and pred_count['count'] >= 10 ):
#                     cv2.putText(back2,casl, (i,j), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
#                     if i <= 20 and  started:
#                         j = j + 26
#                     i = (i + 20)%500
#                     if i == 0:
#                         i = 20
#                     started = True
#                     pred_count['count'] = 0

#             print(pred_count)


#             # Viz probabilities
#             board = prob_viz(res, categories,board, colors)
            

        
#         # Show to screen
#         cv2.imshow('OpenCV Feed', image)
#         cv2.imshow('viz',board)
#         cv2.imshow('Text Converter',back2)

#         # Break gracefully
#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break


#     cap.release()
#     cv2.destroyAllWindows()

NameError: name 'cv2' is not defined

In [51]:
cv2.destroyAllWindows()
cap.release()